<a href="https://colab.research.google.com/github/skybristol/GeoArchiveSummer2021/blob/main/Working_with_a_Trello_archive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The purpose of this notebook is to help understand how someone could interact with an export of a Trello board (working space for tracking tasks) using software code.

One of the tools we used as part of our summer project was [Trello](https://trello.com) for keeping track of our tasks. As part of our overall effort to explore open science practices, this record of what we did as part of the project is important to share. We will archive a final export from Trello in our [OSF workspace](https://osf.io/t6azp/) as part of making even this somewhat mundane aspect of running a project open and transparent. There might be useful information in the record of our tasks for others to build upon.

With its free version, Trello provides an export of a given board as a JSON document. This is available via a URL for board members who are logged in. There are also ways of interfacing programmatically with Trello via an API that could provide other means of getting at the information, and there's even a [Python project](https://pypi.org/project/py-trello/) for working with Trello that could be useful. For our purposes, any interface to what we have in Trello after the project is concluded will be via the archive we save out to OSF. So, we focus our attention here on working with that file.

When our OSF project is made public, we should be able to provide a way for this notebook to call up the export JSON file from the OSF files container programmatically via a web request. For now, you will need to download the JSON document (either from OSF or from Trello itself) and upload it for access from the Colab notebook as our methods below are currently expecting it to be found in the local file system.

In [1]:
import json
import glob
import pandas as pd

In this codeblock we check out local file system for the Trello export we have in our OSF project archive and load it into our notebook from JSON.

In [2]:
trello_export_file = next((i for i in glob.glob("TrelloExport*")), None)
trello_export_data = json.load(open(trello_export_file, "r"))


In [8]:
[i for i in trello_export_data["cards"] if i["badges"]["comments"] > 0]

[{'address': None,
  'attachments': [],
  'badges': {'attachments': 0,
   'attachmentsByType': {'trello': {'board': 0, 'card': 0}},
   'checkItems': 0,
   'checkItemsChecked': 0,
   'checkItemsEarliestDue': None,
   'comments': 1,
   'description': True,
   'due': None,
   'dueComplete': False,
   'fogbugz': '',
   'location': False,
   'start': None,
   'subscribed': True,
   'viewingMemberVoted': False,
   'votes': 0},
  'cardRole': None,
  'checkItemStates': None,
  'closed': False,
  'coordinates': None,
  'cover': {'brightness': 'dark',
   'color': None,
   'idAttachment': None,
   'idPlugin': None,
   'idUploadedBackground': None,
   'size': 'normal'},
  'creationMethod': None,
  'customFieldItems': [],
  'dateLastActivity': '2021-07-10T19:24:01.453Z',
  'desc': "With scientific data, we write up and share detailed documentation about the data and how they were produced to help other people understand and use the data appropriately. There are a number of standards that inform how

The basic task we want to perform is to summarize the actual useful bits of information from our Trello board into a simple data structure that anyone should be able to read and understand. To do that, we need to tease out just the most useful properties from the boards. This basically comes down to the individual cards, which have at least a name and description. Some cards have checklists, which are the individual tasks we performed, and those are useful as well. Some cards have comments in a thread.

In the following codeblock, we loop through the cards first and then tease out checklists and comments, adding four simplified properties to a new set of records for each card.

In [18]:
summary = []
for card in trello_export_data["cards"]:
    summary_card = {
        "name": card["name"],
        "desciption": card["desc"]
    }

    if card["idChecklists"]:
        checklist_id = card["idChecklists"][0]
        checklist = next((i["checkItems"] for i in trello_export_data["checklists"] if i["id"] == checklist_id), None)
        summary_card["tasks"] = [i["name"] for i in checklist]

    if card["badges"]["comments"] > 0:
        summary_card["comments"] = [i["data"]["text"] for i in trello_export_data["actions"] if "card" in i["data"] and i["data"]["card"]["id"] == card["id"] and i["type"] == "commentCard"]

    summary.append(summary_card)

One output we might want is a simple table with the 4 properties in our summary. We can do that in lots of ways, but pulling our list of dictionaries created in the previous step into a Pandas dataframe is a reasonable way to get there. In the following codeblock, we build the dataframe and then output it to a CSV file, which we can then upload to our OSF project archive.

In [20]:
df_summary = pd.DataFrame(summary)
df_summary = df_summary.where(pd.notnull(df_summary), None)
df_summary.to_csv("summary.csv", index=False)
df_summary

,name,desciption,tasks,comments
0,Provide short readme on interfacing with the S...,,None,None
1,Register OSF project for end of project product,,None,None
2,Fill out the readme for our code project,We need to write up some stuff in the Readme f...,None,None
3,Work out how to archive our Slack workspace di...,Slack has an export capability that dumps a di...,[Start wiki page describing why we are archivi...,None
4,Final review of protocol for getting reports f...,We need to do a final group review of the prot...,[Look over all steps and adjust anything neede...,None
5,Finish code in notebook on summarizing Trello ...,,None,None
6,Finish readme for code repo and prepare citati...,,None,None
7,Mint copy of repo via Zenodo for end of projec...,Follow the instructions here - https://guides....,None,None
8,Publish final versions of GeoArchive protocols...,We ended up breaking the protocols up into thr...,None,None
9,Take final snapshot of Trello board once all c...,,None,None
